# Instruções para Carregamento e Ajustes dos Dados
- Utilize o mesmo arquivo utilizado para a previsão Bayesiana

## Proposta para Uso no Dia-a-Dia
A solução será aplicada para prever o desempenho do processo de entrega ao longo do mês, fornecendo estimativas contínuas e ajustáveis. O objetivo é obter uma previsão confiável sobre o percentual de entregas realizadas em menos de 30 minutos, permitindo ações corretivas proativas, se necessário.

### Funcionamento da Solução
Ao longo do mês o modelo preverá a proporção de entregas realizadas abaixo de 30 minutos ao final do mês (prevendo o valor final do processo no mês). Meta: 75% ou mais dos lotes entregues no prazo.

### Ações baseadas na previsão:
- Se % ≥ 75%: Nenhuma ação corretiva necessária.
- Se % < 75%: Aplicar o modelo GBM para identificar os fatores que causam atrasos e tomar medidas específicas.

### Estratégia de Modelagem
- Modelo Bayesiano como Forecast Dinâmico
- O modelo Bayesiano será utilizado para calcular a probabilidade mensal de a % de entregas abaixo de 30 minutos estar abaixo da meta.
- As previsões serão ajustadas continuamente à medida que novos dados de entrega são adicionados diariamente.
- Número de amostras: No uso de um modelo Bayesiano, a quantidade de dados históricos é crucial para obter estimativas precisas dos parâmetros (μ e σ). Idealmente, você deve ter pelo menos 30 a 50 pontos de dados para garantir que as distribuições posteriores sejam confiáveis e bem definidas.
- Tendência e variação: Quanto mais dados você tiver, mais facilmente o modelo poderá capturar as tendências e variações sazonais no processo de entrega, proporcionando previsões mais estáveis e precisas ao longo do tempo.
- Distribuição Log-Normal: O modelo considera a distribuição log-normal, que é a distribuição esperada para processos que envolvem tempos de entrega, como em casos urgentes de medicamentos (por exemplo, pacientes com dor). A distribuição log-normal é ideal para representar esses tempos, que geralmente possuem uma cauda mais pesada e são assimétricos.

### Integração com Gradient Boosting Machines (GBM)
A previsão diária gerada pelo modelo Bayesiano indicará a necessidade de uma análise mais apronfundada. O GBM será usado para refinar a previsão e identificar as principais causas de atrasos. 
Features que podem ser usadas no GBM: Dias da Semana, Hora de entrega, Hora de solicitação, Usuário que realizou a entrega, entre outras...

### Análise e Ação
A combinação dos modelos permite:
- Previsão Precisa do Desempenho: O modelo Bayesiano fornece uma estimativa inicial confiável.
- Identificação das Causas: O GBM identifica as variáveis mais influentes nos atrasos e auxilia na formulação de estratégias de melhoria.

### Vantagens do Modelo Proposto
Previsão Contínua: A abordagem Bayesiana permite atualizações diárias, garantindo previsões mais precisas à medida que o mês avança.
Insights Aprofundados: O GBM complementa a análise, ajudando a diagnosticar problemas específicos no processo.
Tomada de Decisão Proativa: Com base na previsão, a equipe pode atuar antes que a meta de 75% seja comprometida.
Essa solução combina a adaptabilidade do modelo Bayesiano com o poder analítico do GBM, garantindo previsões confiáveis e ações corretivas orientadas por dados.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Exemplo de dados (substitua com os dados reais)
# As variáveis precisam ser ajustadas conforme a estrutura do seu dataset
df = pd.read_csv('seu_arquivo_de_dados.csv')

# Colunas importantes
features = ['']  # Defina quais colunas utilizará 
target = 'Proporcao_Entrega_Em_Tempo'

# Preprocessamento: converter variáveis categóricas para numéricas
df = pd.get_dummies(df, columns= [features])

# Separar as features (X) e o alvo (y)
X = df[features]
y = df[target]

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinamento do modelo GBM com XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.01, max_depth=6)
model.fit(X_train, y_train)

# Previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliação do modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

# Importância das variáveis
importancia = model.feature_importances_

# Visualização da importância das variáveis
plt.figure(figsize=(10, 6))
sns.barplot(x=importancia, y=features)
plt.title('Importância das Variáveis - Modelo GBM')
plt.xlabel('Importância')
plt.ylabel('Features')
plt.show()

# Análise de desempenho
accuracy = accuracy_score(y_test, np.round(y_pred))
print(f"Accuracy (classificação por meta de 75%): {accuracy}")